In [1]:
# picodet from scratch

In [2]:
import sys
sys.path.insert(0, "..")

from src.esnet import ESNet

In [3]:
from pathlib import Path
from typing import Sequence, Optional, Union
from pydantic.dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
from mmcv.cnn import ConvModule
from mmcv.runner import BaseModule
from mmdet.models.utils import make_divisible

## Backbone: ESNet

In [4]:
esnet = ESNet()

/root/.cache/pypoetry/virtualenvs/dev-sZr6PvLR-py3.9/lib/python3.9/site-packages/mmcv/cnn/bricks/hsigmoid.py:31: UserWarning: In MMCV v1.4.4, we modified the default value of args to align with PyTorch official. Previous Implementation: Hsigmoid(x) = min(max((x + 1) / 2, 0), 1). Current Implementation: Hsigmoid(x) = min(max((x + 3) / 6, 0), 1).
  warnings.warn(


In [5]:
esnet.out_ixs

[2, 9, 12]

In [6]:
esnet.stage_out_channels
test_input = torch.from_numpy(np.random.rand(1, 3, 320, 320).astype(np.float32))
test_outputs = esnet(test_input)
print([a.shape for a in test_outputs])

[torch.Size([1, 96, 40, 40]), torch.Size([1, 192, 20, 20]), torch.Size([1, 384, 10, 10])]
